### Langchain Agent example with OCI Meta model
Using the old version of tools


In [1]:
%%capture
!pip install langchain_community
!pip install tavily-python
!pip install duckduckgo-search

In [3]:
from langchain_community.chat_models import ChatOCIGenAI
from langchain_core.messages import HumanMessage
from langchain.agents import initialize_agent, Tool
import time
from tavily import TavilyClient
import os

# Tavily config API
tavily_client = TavilyClient(api_key='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')

# Função para realizar a busca
def search_web(query):
    try:
        response = tavily_client.search(query=query,include_domains=["webmd.com"])
        return response
    except Exception as e:
        print(f"Ocorreu um erro ao realizar a busca: {e}")
        return None


# Configura a tool, ela pode ser qualquer função python
# não funciona bem com funções com mais de um argumento
search_tool = Tool(
    name="Pesquisa na Web",
    func=search_web,
    description="Realiza uma pesquisa na web para obter informações sobre saúde e bem estar."
)

# Configura o modelo OCI GenAI
COMPARTMENT_ID = "ocid1.compartment.oc1..aaaaaaaael2x4x2flfdgrteymct63e73zghik5gbyzd2zsww7ihzkfldpdza"
AUTH_TYPE = "API_KEY"
CONFIG_PROFILE = "DEFAULT"
endpoint = "https://inference.generativeai.sa-saopaulo-1.oci.oraclecloud.com"

# initialize interface
chat = ChatOCIGenAI(
  model_id="ocid1.generativeaimodel.oc1.sa-saopaulo-1.amaaaaaask7dceyarsn4m6k3aqvvgatida3omyprlcs3alrwcuusblru4jaa",
  service_endpoint=endpoint,
  compartment_id=COMPARTMENT_ID,
  provider="meta",
  model_kwargs={
    "temperature": 1,
    "max_tokens": 600,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "top_p": 0.75
  },
  auth_type=AUTH_TYPE,
  auth_profile=CONFIG_PROFILE
)
tools=[search_tool]
tool_names = ['search_tool']

# Inicializa o agent (é possível utilizar o pre build react agent no langgraph também)
agent = initialize_agent(
    tools=[search_tool],
    llm=chat,
    agent="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True
)

############# Input
user_input = "Estou com dor no joelho"

prompt = f"""Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

    {tools}

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

    Begin! Remember to answer as a compansionate medical professional when giving your final answer.
    Do always think and answer in portuguese
    Query: {user_input}"""


# Executar o agente 
response = agent.invoke(prompt)


print("**************************Resultado do Chat**************************")
print(response['output'])




> Entering new AgentExecutor chain...
Parece que o paciente está apresentando dor no joelho, o que pode ser causado por uma variedade de fatores, incluindo lesões, condições médicas ou problemas ortopédicos. É importante investigar mais a fundo para determinar a causa exata da dor.

Action: Pesquisa na Web
Action Input: causas de dor no joelho

Observation: {'query': 'causas de dor no joelho\n', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': '15 Conditions That Can Cause Leg Pain - WebMD', 'url': 'https://www.webmd.com/pain-management/ss/slideshow-leg-pain-causes', 'content': 'If that doesnâ\x80\x99t work, your doctor might give you medicine to treat the problem or help with pain. Your doctor will treat the condition thatâ\x80\x99s causing it and give you medicine for pain if you need it. That puts pressure on the nerves in the area and can cause pain, tingling, numbness, or weakness in your legs. This is a common condition that affects your joints a